In [1]:
from gensim.summarization import summarize
from newspaper import Article
from model import Model
from utils import build_dict, build_dataset, batch_iter
from rouge import Rouge
import pickle
import pandas as pd
import re
import tensorflow as tf
import random 
import nltk 
from nltk.tag import pos_tag
from nltk import FreqDist
from nltk.tokenize import word_tokenize

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# 지문 생성 및 문제 생성을 위한 텍스트 파일 생성 
def crawl_news(url, raw_txt_name, num_word):
    # URL 기반 크롤링, 언어는 영어로 설정
    news = Article(url, language = 'en')
    news.download()
    # HTML 파싱 적용
    news.parse()
    # 지정된 단어 갯수를 기준으로 Textrank를 적용하여 파싱된 텍스트를 요약 (Rule base)
    text = summarize(news.text, word_count = num_word) # 400 ~ 800자 기준으로 하기
    # 파싱된 HTML에서 뉴스 기사에 해당하는 부분 가져오기
    title = news.title
    # 파싱된 HTML에서 뉴스 본문에 해당하는 부분 가져오기
    title = title.lower()
    # 뉴스 제목 중에서 괄호가 포함되어 있는 경우, 해당 부분을 제거하기
    title = re.sub('\([^)]*\)', '', title) # 괄호 + 괄호 안 내용 제거 
    
    # 크롤링한 raw data를 별도로 저장
    with open(raw_txt_name, 'w', encoding = 'utf-8') as f: 
        f.write(text)
    # 크롤링한 뉴스 기사 문단 분리 없이 변수 할당
    text = re.sub('\n', ' ', text)

    return text, title

In [3]:
# 소설 지문 생성을 위한 함수를 별도로 정의 
def bf_create_novel_topic(raw_txt_name, raw_title_name, prepro_txt_name):
    
    # '문장 번호: 문장' 형식의 저장을 위해 변수 생성  
    test_dict = {}
    
    # 저장했었던 크롤링 본문 데이터 불러오기
    with open(raw_txt_name, 'r', encoding = 'utf-8') as f:
        # raw data 상태에서 전처리를 통해 모델에 들어갈 수 있는 데이터셋으로 변형
        for idx, text in enumerate(f.readlines()):
            text = text.lower() # 소문자로 통일
            text = re.sub('[.]', '', text) # 일단 온점 모두 제거
            text = re.sub('[,]', ' ,', text) # 반점 하나의 단어로 인식
            text = re.sub("\’s",' \'s', text) # 소유격도 하나의 단어로 인식
            text = re.sub("[0-9]", '#', text) # 숫자를 #로 치환
            #text = re.sub("'.+'", '', text) # 작은 따옴표 안 고유명사 제거 
            text = re.sub('[“”]', '', text) # 큰 따옴표 제거 
            text = re.sub('\n', '', text) # 줄 바꿈표 제거 
            print(idx, text)
            
            test_dict[idx] = text
    # 전처리된 데이터를 별도의 파일로 저장    
    with open(prepro_txt_name, 'w', encoding = 'utf-8') as f:
        # 줄 바꿈을 기준으로 문장을 구분하여 저장 
        for i in range(len(test_dict)):
            test_dict[i] = test_dict[i] + " " + "." + '\n'
            f.write(test_dict[i])
    # 소설의 경우 해당 지문을 위한 별도의 제목 데이터를 불러오기
    with open(raw_title_name, 'r', encoding = 'utf-8') as f:
        title = f.read()
    
    
    
    return test_dict, title
    

In [4]:
# 일반 주제 지문 생성을 위한 함수를 별도로 정의
def bf_create_topic(raw_txt_name, prepro_txt_name):
    # '문장 번호: 문장' 형식의 저장을 위해 변수 생성 
    test_dict = {}

    # 저장했었던 크롤링 본문 데이터 불러오기
    with open(raw_txt_name, 'r', encoding = 'utf-8') as f:
        # raw data 상태에서 전처리를 통해 모델에 들어갈 수 있는 데이터셋으로 변형
        for idx, text in enumerate(f.readlines()):
            # print(idx, text)
            text = text.lower() # 소문자로 통일
            text = re.sub('[.]', '', text) # 일단 온점 모두 제거
            text = re.sub('[,]', ' ,', text) # 반점 하나의 단어로 인식
            text = re.sub("\’s",' \'s', text) # 소유격도 하나의 단어로 인식
            text = re.sub("[0-9]", '#', text) # 숫자를 #로 치환
            #text = re.sub("'.+'", '', text) # 작은 따옴표 안 고유명사 제거 
            text = re.sub('[“”]', '', text) # 큰 따옴표 제거 
            text = re.sub('\n', '', text) # 줄 바꿈표 제거 
            print(idx, text)
        
            # 사전에 저장하기
            test_dict[idx] = text
            
    # 전처리된 데이터를 별도의 파일로 저장    
    with open(prepro_txt_name, 'w', encoding = 'utf-8') as f: #
        # 줄 바꿈을 기준으로 문장을 구분하여 저장 
        for i in range(len(test_dict)):
            test_dict[i] = test_dict[i] + " " + "." + '\n'
            f.write(test_dict[i])
    
    return test_dict 

In [5]:
# 텍스트 요약 학습 데이터 불러오기 
def shake_bot_topic(filename, vaild_article_path):
    # 설정된 세팅 값 불러오기 
    with open("args.pickle", "rb") as f:
        args = pickle.load(f)
    
    # 실행시 최초의 상태로 초기화하여 모델 재사용이 가능하도록 함
    tf.reset_default_graph()
    # 학습된 단어 사전과 본문 최대 길이와 요약 최대 길이 설정 값을 불러옴
    print("셰봇이 단어 사전 불러오는 중...")
    word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
    # 불러온 설정 값에 따라서 검증 데이터셋을 생성
    print("셰봇이 기본 설정 값 초기화 중...")
    valid_x = build_dataset("valid", vaild_article_path, word_dict, article_max_len, summary_max_len, args.toy)
    valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]

    with tf.Session() as sess:
        # 학습된 체크포인트와 모델을 불러오기 
        print("Loading saved model...")
        model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True) # 검증 단계
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state("./saved_model/")
        saver.restore(sess, ckpt.model_checkpoint_path)
        
        # 검증을 진행할 배치 사이즈 설정
        batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)
        
        # 검증 단계인 만큼 인코더 인풋 부분만 고려하여 학습 진행
        print("셰봇이 수능 문제 생성 준비 중.. {}...".format(filename))
        for batch_x, _ in batches:
            # 문장 -> 문장 예측이 가능하도록 데이터 분할 
            batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]
            # 검증을 위한 배치에 따라 모델 및 데이터 셋 설정 준비 
            valid_feed_dict = {
                model.batch_size: len(batch_x),
                model.X: batch_x,
                model.X_len: batch_x_len,
            }
            # Prediction을 목적으로 학습된 모델을 검증을 위한 설정 값을 적용하여 예측 결과 생성
            prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
            # 인덱스 -> 단어 사전을 통해 학습된 결과를 단어로 표현.
            prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
            
            # 생성된 예측 결과물을 문장 단위로 저장 
            with open(filename, "w") as f:
                for line in prediction_output:
                    summary = list()
                    for word in line:
                        if word == "</s>":
                            break
                        if word not in summary:
                            summary.append(word)
                    print(" ".join(summary), file=f)
                    
            #with open('./rouge_test/sys_dir/{}.txt'.format(filename), "w") as f:
                #for line in prediction_output:
                    #summary = list()
                    #for word in line:
                        #if word == "</s>":
                            #break
                        #if word not in summary:
                            #summary.append(word)
                    #print(" ".join(summary), file=f)

        print('셰봇이가 5지 선다로 쓸만한 친구들을 추려냈어요.. {}...'.format(filename))
        
        


In [6]:
#shake_bot_topic('CSAT.001')

In [7]:
# Rouge test 준비
#Rouge155.convert_summaries_to_rouge_format('./rouge_test/sys_dir/sys_input', './rouge_test/sys_dir/sys_output')
#Rouge155.convert_summaries_to_rouge_format('./rouge_test/md_dir/md_input', './rouge_test/md_dir/md_input')

#system_dir = './rouge_test/sys_dir'
#model_dir =  './rouge_test/md_dir'
#system_filename_pattern = 'CSAT.(\d+).txt'  ## CSAT.001.txt
#model_filename_pattern = 'CSAT.[A-Z].#ID#.txt' ## CSAT.A.001.txt
# config_file_path = './rouge_test/config'

#Rouge155.write_config_static(
#    system_dir, system_filename_pattern,
#    model_dir, model_filename_pattern,
#    config_file_path)

In [8]:
# 학습이 잘된 문장을 선별하여 선지로 만들기 위해 함수 생성
def for_rouge_test(summy, label):
    rouge_dict = {}
    rouge_list = []
    rouge = Rouge()
    
    # 학습 결과로 예측한 결과물을 불러오기 
    with open(summy, 'r', encoding = 'utf-8') as f:
            
        for idx, line in enumerate(f.readlines()):
            
            # 실제 문장 - 예측 결과에서 학습의 정도를 Rouge-r과 Rouge-p로 검증
            scores = rouge.get_scores(line, label[idx])
            
            p_score = scores[0]["rouge-1"]["p"] 
            r_score = scores[0]["rouge-1"]["r"] 
            
            # Rouge-r 사용
            r_idx_score = (idx, r_score)            
            
            rouge_list.append(r_idx_score)
            # print(rouge_list)
            
            rouge_dict[idx] = line
        
        # 내림차순으로 가장 점수가 높은 문장을 순서로 리스트 배열
        sort_rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[-1], reverse=True)
        # 상위 4개 문장 추출하기 
        sort_rouge_list = sort_rouge_list[:4]
        
        question_list = []
        
        for ix in sort_rouge_list:
            
            question_list.append(rouge_dict[ix[0]])
        
    # 상위 4개 문장에 대해서 선지로 활용할 수 있도록 리턴 
    return question_list
            
            

In [9]:
# 학습 결과물 중 <unk> 부분 처리 
def delete_unk(content, unk_sentence):
    
    container = []
    
    text = open(content, 'r', encoding = 'utf-8').read()
    
    # 각 단어에 대해서 품사 태깅
    tagged_list = pos_tag(word_tokenize(text))
    
    # 명사 부분만 활용
    nnp_list = [t[0] for t in tagged_list if (t[1] == "NNP") or (t[1] == "NN")]
    
    fd_names = FreqDist(nnp_list)
    
    # 빈도수 기반으로 unk 처리하기 
    freq = fd_names.most_common(2)
    
    #for tagged in tagged_list:
        
        #if tagged[1] == 'NN' or 'NNP':
            
            #container.append(tagged[0]) # 단어만 추출
    
    
    # unk 부분을 처리하는 방법 중 하나로 최고 빈도수를 기반으로 하는 방식
    replace_sentence = re.sub('< unk >', freq[0][0], unk_sentence)
    
    # 다중 숫자에 대한 표현은 Several로 대체 
    replace_sentence = re.sub('# nd', 'Several', replace_sentence)
    
    # 불필요한 줄 바꿈표 제거 
    replace_sentence = re.sub('\n', '', replace_sentence)
    
    return replace_sentence
     
    
    
    
    
            
            
            
            
            
    

In [10]:
# Tess 소설 지문 사용하기 
num_word = 600
raw_title_name = 'novel_title_18.txt'
raw_txt_name = 'novel_letter_18.txt'
prepro_txt_name = 'prepro_18.txt'
learned_txt_name = 'topic_18.txt'

def topic_18(raw_txt_name, raw_title_name, prepro_txt_name, learned_txt_name, num_word):
    
    # 현재까지는 소설의 경우 사전 수집한 데이터 활용
    text = open(raw_txt_name, 'r', encoding = 'utf-8').read()
    text = re.sub('\n', '', text)
    
    # 문장 별 딕셔너리와 제목 추출
    test_dict, title = bf_create_novel_topic(raw_txt_name, raw_title_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_topic(learned_txt_name, prepro_txt_name) 
    
    # Rouge Score에 따라 선지 생성 
    question_list = for_rouge_test(learned_txt_name, test_dict)
    
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        replace_sentence = replace_sentence.capitalize() + "."
        
        question_list[idx] = replace_sentence
    
    # 제목 부분도 정답 선지로 활용할 수 있도록 전처리
    title = re.sub('\n+', '', title)
    title = title.strip()
    title = title.capitalize() + "."
    question_list.append(title)
    
    # 생성 시마다 정답의 번호가 바뀔 수 있도록 랜덤 셔플 적용
    random.shuffle(question_list)
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 전처리 및 넘버링이 완료된 선지를 최종 문제로 활용
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('18. 다음 필자가 전달하려는 것으로 가장 적절한 것은?')
    print('-----------------------------------')
    print(text) # 지문 500자
    print('\n')
    for question in question_list:
        print(question)    

if __name__ == "__main__":
    
    topic_18(raw_txt_name, raw_title_name, prepro_txt_name, learned_txt_name, num_word)
    
        
        

0 dear tess -
1 
2 i write these few lines hoping they will find you well , as they leave me at present , thank god for it 
3 dear tess , we are all glad to hear that you are going really to be married soon 
4 but with respect to your question , tess , j say between ourselves , quite private but very strong , that on no account do you say a word of your bygone trouble to him 
5 i did not tell everything to your father , he being so proud on account of his respectability , which , perhaps , your intended is the same
6 many a woman—some of the highest in the land—have had a trouble in their time; and why should you trumpet yours when others don’t trumpet theirs? 
7 no girl would be such a fool , specially as it is so long ago , and not your fault at all
8 i shall answer the same if you ask me fifty times
9 besides , you must bear in mind that , knowing it to be your childish nature to tell all that 's in your heart—so simple!—j made you promise me never to let it out by word or deed , ha

In [10]:
# Opinion Section에 해당하는 기사를 활용하여, 주장 문제 만들기 

url = 'https://www.nytimes.com/2019/12/23/science/earth-science-diversity-education.html'
# 지문의 길이는 뉴스 기사의 길이에 따라 유동적으로 설정하기 
num_word = 400
raw_txt_name = 'CSAT_20.txt'
prepro_txt_name = 'prepro_20.txt'
learned_txt_name = 'topic_20.txt'

def topic_20(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title = crawl_news(url, raw_txt_name, num_word) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_topic(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_topic(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    # Rouge Score에 따라 선지 생성 
    question_list = for_rouge_test(learned_txt_name, test_dict)
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        replace_sentence = replace_sentence.capitalize() + "."
        
        question_list[idx] = replace_sentence
    
    # 제목 부분도 정답 선지로 활용할 수 있도록 전처리
    title = re.sub('\n+', '', title) 
    title = title.strip()
    title = title.capitalize() + "."
    question_list.append(title)
    
    # 생성 시마다 정답의 번호가 바뀔 수 있도록 랜덤 셔플 적용
    random.shuffle(question_list)
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 전처리 및 넘버링이 완료된 선지를 최종 문제로 활용
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('20. 다음 글에서 필자가 주장하는 바로 가장 적절한 것은?')
    print('-----------------------------------')
    print(text) # 지문 500자
    print('\n')
    for question in question_list:
        print(question)

if __name__ == "__main__":
    
    topic_20(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word)

0 when arianna varuolo-clarke was growing up , her favorite evenings were spent watching the weather channel with her grandfather
1 she wanted to chase thunderstorms and understand where tornadoes came from , she said
2 she decided to become an atmospheric scientist
3 in #### , she landed an internship at the national center for atmospheric research as a college sophomore , and quickly realized that her path as a woman of color would not be easy
4 you’d walk through the halls and it 's a lot of old white men , ms varuolo-clarke said
5 still , she pushed forward and began her phd in atmospheric science at columbia university last year
6 the field 's lack of diversity gained new urgency in may when her graduate student cohort was targeted with a series of racist emails
7 the messages , sent to affiliates of the lamont-doherty earth observatory at columbia by a person outside the community , said that black people were genetically inferior and did not belong in academia
8 it was hurtful a

In [68]:
# 일반 주제 지문 생성
url = 'https://www.nytimes.com/2019/10/25/your-money/franchise.html?searchResultPosition=2'
# 지문의 길이는 뉴스 기사의 길이에 따라 유동적으로 설정하기 
num_word = 300
raw_txt_name = 'CSAT_21.txt'
prepro_txt_name = 'prepro_21.txt'
learned_txt_name = 'topic_21.txt'

def topic_21(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title = crawl_news(url, raw_txt_name, num_word) # url과 raw_txt_name 파일명 입력    
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_topic(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_topic(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    # Rouge Score에 따라 선지 생성 
    question_list = for_rouge_test(learned_txt_name, test_dict)
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        replace_sentence = replace_sentence.capitalize() + "."
        
        question_list[idx] = replace_sentence
  
    # 제목 부분도 정답 선지로 활용할 수 있도록 전처리
    title = re.sub('\n+', '', title)
    title = title.strip()
    title = title.capitalize() + "."
    question_list.append(title)
    
    # 생성 시마다 정답의 번호가 바뀔 수 있도록 랜덤 셔플 적용
    random.shuffle(question_list)
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 전처리 및 넘버링이 완료된 선지를 최종 문제로 활용
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('21. 다음 글의 요지로 가장 적절한 것은?')
    print('-----------------------------------')
    print(text) # 지문 400자
    print('\n')
    for question in question_list:
        print(question)
    
    
    
    
    
    
    

if __name__ == "__main__":
    
    topic_21(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word)

0 if someone is inclined to buy a franchise , whether for the independence or the wealth , a cleareyed view of what is being pitched is important
1 here are some questions to consider
2 pick a winner
3 for starters , analysts say , investors should ask whether the underlying industry is growing
4 but even in growth areas , not every franchise is a winner
5 franchising , like the economy , is cyclical
6 data shows that the top growth areas for franchising include fitness centers , children 's educational programs and the general health sector; food , as a broad category , is ebbing
7 potential buyers should look carefully at federal disclosure forms that are required for franchises , paying particular attention to item ## , which addresses the return on investment and the unit economics , said jeff johnson  , founder and chief executive of the franchise research institute
8 the second big factor is whether branding will matter
9 darrell johnson of frandata said most people know their pr

In [69]:
# 일반 제목 지문 생성
url = 'https://www.nytimes.com/2019/10/25/your-money/franchise.html?searchResultPosition=2'
# 지문의 길이는 뉴스 기사의 길이에 따라 유동적으로 설정하기 
num_word = 800
raw_txt_name = 'CSAT_22.txt'
prepro_txt_name = 'prepro_22.txt'
learned_txt_name = 'topic_22.txt'

def topic_22(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word):
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title = crawl_news(url, raw_txt_name, num_word) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_topic(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_topic(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    # Rouge Score에 따라 선지 생성 
    question_list = for_rouge_test(learned_txt_name, test_dict)
    
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        replace_sentence = replace_sentence.title() + "."
        
        question_list[idx] = replace_sentence
    
     # 제목 부분도 정답 선지로 활용할 수 있도록 전처리
    title = re.sub('\n+', '', title)
    title = title.strip()
    title = title.title() + "."
    question_list.append(title)
    
    # 생성 시마다 정답의 번호가 바뀔 수 있도록 랜덤 셔플 적용
    random.shuffle(question_list)
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 전처리 및 넘버링이 완료된 선지를 최종 문제로 활용
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('22. 다음 글의 제목으로 가장 적절한 것은?')
    print('-----------------------------------')
    print(text) 
    print('\n')
    for question in question_list:
        print(question)
    
    
    

if __name__ == "__main__":
    
    topic_22(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word)

0 if someone is inclined to buy a franchise , whether for the independence or the wealth , a cleareyed view of what is being pitched is important
1 here are some questions to consider
2 pick a winner
3 for starters , analysts say , investors should ask whether the underlying industry is growing
4 but even in growth areas , not every franchise is a winner
5 franchising , like the economy , is cyclical
6 data shows that the top growth areas for franchising include fitness centers , children 's educational programs and the general health sector; food , as a broad category , is ebbing
7 potential buyers should look carefully at federal disclosure forms that are required for franchises , paying particular attention to item ## , which addresses the return on investment and the unit economics , said jeff johnson  , founder and chief executive of the franchise research institute
8 the second big factor is whether branding will matter
9 darrell johnson of frandata said most people know their pr

In [12]:
# 일반 주제 지문 생성
url = 'https://www.nytimes.com/2019/10/25/your-money/franchise.html?searchResultPosition=2'
# 기사 길이에 맞추어 유동적으로 설정
num_word = 600
raw_txt_name = 'CSAT_23.txt'
prepro_txt_name = 'prepro_23.txt'
learned_txt_name = 'topic_23.txt'

def topic_23(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title = crawl_news(url, raw_txt_name, num_word) # url과 raw_txt_name 파일명 입력    
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_topic(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_topic(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    # Rouge Score에 따라 선지 생성 
    question_list = for_rouge_test(learned_txt_name, test_dict)
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        replace_sentence = replace_sentence.capitalize() + "."
        
        question_list[idx] = replace_sentence
  
    # 제목 부분도 정답 선지로 활용할 수 있도록 전처리
    title = re.sub('\n+', '', title)
    title = title.strip()
    title = title.capitalize() + "."
    question_list.append(title)
    
    # 생성 시마다 정답의 번호가 바뀔 수 있도록 랜덤 셔플 적용
    random.shuffle(question_list)
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 전처리 및 넘버링이 완료된 선지를 최종 문제로 활용
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('23. 다음 글의 주제로 가장 적절한 것은?')
    print('-----------------------------------')
    print(text) # 지문 800자
    print('\n')
    for question in question_list:
        print(question)
    
    
    
    
    
    
    

if __name__ == "__main__":
    
    topic_23(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word)

0 if someone is inclined to buy a franchise , whether for the independence or the wealth , a cleareyed view of what is being pitched is important
1 here are some questions to consider
2 pick a winner
3 for starters , analysts say , investors should ask whether the underlying industry is growing
4 but even in growth areas , not every franchise is a winner
5 franchising , like the economy , is cyclical
6 data shows that the top growth areas for franchising include fitness centers , children 's educational programs and the general health sector; food , as a broad category , is ebbing
7 potential buyers should look carefully at federal disclosure forms that are required for franchises , paying particular attention to item ## , which addresses the return on investment and the unit economics , said jeff johnson  , founder and chief executive of the franchise research institute
8 the second big factor is whether branding will matter
9 darrell johnson of frandata said most people know their pr

In [38]:
# 일반 제목 지문 생성
url = 'https://www.nytimes.com/2019/10/25/your-money/franchise.html?searchResultPosition=2'
# 뉴스 기사 길이에 따라 유동적으로 설정
num_word = 400
raw_txt_name = 'CSAT_24.txt'
prepro_txt_name = 'prepro_24.txt'
learned_txt_name = 'topic_24.txt'

def topic_24(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title = crawl_news(url, raw_txt_name, num_word) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_topic(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_topic(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    # Rouge Score에 따라 선지 생성 
    question_list = for_rouge_test(learned_txt_name, test_dict)
    
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        replace_sentence = replace_sentence.title() + "."
        
        question_list[idx] = replace_sentence
    
     # 제목 부분도 정답 선지로 활용할 수 있도록 전처리
    title = re.sub('\n+', '', title)
    title = title.strip()
    title = title.title() + "."
    question_list.append(title)
    
    # 생성 시마다 정답의 번호가 바뀔 수 있도록 랜덤 셔플 적용
    random.shuffle(question_list)
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 전처리 및 넘버링이 완료된 선지를 최종 문제로 활용
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('24. 다음 글의 제목으로 가장 적절한 것은?')
    print('-----------------------------------')
    print(text) # 지문 800자
    print('\n')
    for question in question_list:
        print(question)
    
    
    
    
    
    
    

if __name__ == "__main__":
    
    topic_24(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word)
    
        

0 if someone is inclined to buy a franchise , whether for the independence or the wealth , a cleareyed view of what is being pitched is important
1 here are some questions to consider
2 pick a winner
3 for starters , analysts say , investors should ask whether the underlying industry is growing
4 but even in growth areas , not every franchise is a winner
5 franchising , like the economy , is cyclical
6 data shows that the top growth areas for franchising include fitness centers , children 's educational programs and the general health sector; food , as a broad category , is ebbing
7 potential buyers should look carefully at federal disclosure forms that are required for franchises , paying particular attention to item ## , which addresses the return on investment and the unit economics , said jeff johnson  , founder and chief executive of the franchise research institute
8 the second big factor is whether branding will matter
9 darrell johnson of frandata said most people know their pr